In [46]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [47]:
spark=SparkSession.builder.appName("clusteringProject").getOrCreate()

In [48]:
df=spark.read.csv("hack_data.csv",inferSchema=True,header=True)

In [49]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [50]:
df.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [51]:
df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [52]:
df.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [53]:
feature_list=['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted',
'WPM_Typing_Speed']

In [54]:
df=df[feature_list]

In [55]:
assembler=VectorAssembler(inputCols=feature_list,outputCol="featureVector")

In [56]:
scaler=StandardScaler(inputCol="featureVector",outputCol="scaledFeature")

In [57]:
df=assembler.transform(df)

In [58]:
df=scaler.fit(df).transform(df)

In [59]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- featureVector: vector (nullable = true)
 |-- scaledFeature: vector (nullable = true)



In [60]:
kmeans=KMeans(featuresCol="featureVector",k=2)

In [61]:
model=kmeans.fit(df)

In [62]:
model.clusterCenters()

[array([4.00060976e+01, 8.59157866e+02, 5.36585366e-01, 6.92634146e+00,
        1.32134146e+01, 4.68612195e+01]),
 array([ 20.36470588, 364.22370588,   0.48823529,   3.64952941,
          8.54705882,  67.45364706])]

In [63]:
result=model.transform(df)

In [64]:
model.computeCost(result)

6915193.171893887

In [65]:
result.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- featureVector: vector (nullable = true)
 |-- scaledFeature: vector (nullable = true)
 |-- prediction: integer (nullable = true)



In [66]:
result.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  170|
|         0|  164|
+----------+-----+

